In [17]:
import pandas as pd
import numpy as np

data = pd.read_csv("/home/j-k13b204/S13P31B204/model_test/Code/csv_collection/intent_grouping.csv")
data.head()

,text,expression,intent_group
0,앞에서도 말씀드렸지만 저는 중국에서 일 년 정도 유학 생활을 했었습니다.,c_private,Communication
1,아 네 저는 제 나름대로 스트레스 해소법은 어 첫 번째로는 등산을 좋아합니다.,c_private,Communication
2,직무 지식은 본인이 조직의 존재감을 나타내고 업무 수행하는 데 있어서 가장 기본적인...,c_value,Communication
3,그래서 결국은 지금 전역을 하고 지금 이 자리에서 면접을 보고 있는데 저의 선택이 ...,c_sincere_co,Communication
4,어 저는 성격이 에 조금 급한 부분이 있어요.,c_person,Communication


In [18]:
data['text_len'] = data['text'].astype(str).apply(len)

data_len = data['text_len'].mean()
data_len

83.98805198004557

In [19]:
data['intent_group'].value_counts().reset_index()

,intent_group,count
0,Communication,19963
1,Integrity,4479
2,Job_Competency,3777
3,Adaptability,2549
4,Teamwork_Leadership,1706


In [20]:
# text 최대길이 확인

max_len = 0
max_text = ""

for text in data['text'] : 
    length = len(str(text))

    if length > max_len : 
        max_len = length
        max_text = text


print(max_len)
print(max_text)


502
저는 기본적으로 그 대인관계도 좀 좋은 편이고 어 사람들과 어떤 활동을 같이 하는 거 이런 거 굉장히 좋아해서   음 소통 능력이 좀 뛰어난 편이라고 생각을 하는데 아 이 지금 제가 가진 것보다 더 많은 소통 능력을 키우고   그것을 기반으로 해서 어떤 능력을 발휘를 해야 한다면 저는 다양한 모임을 나가서 여러 가지 상황과 여러 가지 이야기 그리고 다양성을 가진 사람들과   음 접하고 어떤 활동을 하면서 자연스럽게 그 소통을 할 수 있는 노하우나   그런 능력을 좀 함양을 할 수 있었으면 하구요. 그 외에 그 어떤 주제를 놓고   아 많은 이야기를 자기 의견을 적극적으로 개진하면서 많은 이야기를 나눠보는 그런 주제 맞춤형 대화 프로그램 같은 것도 소통 능력을 키울 수   있는 방법이라고 생각해서 그런 것도 해보고 싶다는 생각이 들구요. 어 소통에서 가장 중요한 건 내가 어떤 의견을   잘 표현하는 것도 중요하지만 상대방의 그 말을 정확하게 이해하고 받아들이는 자세도 너무 중요하기 때문에


In [21]:
data['intent_group'].unique()

array(['Communication', 'Teamwork_Leadership', 'Integrity',
       'Adaptability', 'Job_Competency'], dtype=object)

In [ ]:
label_dict = {
    'Communication': 0,
    'Teamwork_Leadership': 1,
    'Integrity': 2,
    'Adaptability': 3,
    'Job_Competency': 4
}

data['label'] = data['intent_group'].map(label_dict)

data['label'].unique()

array([0, 1, 2, 3, 4])

## 모델 제작

In [18]:
import torch
import torch.nn as nn
from torch.optim import AdamW    
from torch.utils.data import Dataset, DataLoader

import numpy as np
from tqdm import tqdm
from transformers import get_cosine_schedule_with_warmup 

from kobert_transformers import get_kobert_model, get_tokenizer

# BERT 모델/토크나이저
bertmodel = get_kobert_model()
tokenizer = get_tokenizer()

device = torch.device("cuda")


In [19]:
# 하이퍼파라미터
NUM_LABELS   = 5   # Communication, Integrity, Job_Competency, Adaptability, Teamwork_Leadership
MAX_LEN      = 502
BATCH_SIZE   = 32
EPOCHS       = 10
LR           = 3e-5
WEIGHT_DECAY = 0.01
DROPOUT      = 0.3
PATIENCE     = 2  # validation이 2epoch동안 증가 안 할시 멈춰

In [20]:
import os, random
import numpy as np

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(42)

# MAX_LEN 안전선
assert MAX_LEN <= 512


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# dataset class 구성

class BertClassifier(nn.Module) : 
    def __init__(self, 
                 bert,
                 hidden_size : int = 768,
                 num_classes : int = 5,
                 dr_rate : float = 0.3,
                 class_weights: torch.Tensor | None = None) :
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=dr_rate) if dr_rate and dr_rate > 0 else nn.Identity()
        self.classifier = nn.Linear(hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights) if class_weights is not None else nn.CrossEntropyLoss()
    
    def forward(self,
                input_ids: torch.Tensor, # 문장을 숫자로 바꾼 결과
                attention_mask: torch.Tensor | None = None, # 실제 단어와 pad를 구분
                token_type_ids: torch.Tensor | None = None, # 세그먼트 정보
                labels: torch.Tensor | None = None):
        
        outputs = self.bert(input_ids = input_ids,
                            attention_mask = attention_mask,
                            token_type_ids = token_type_ids)
        
        # HF 출력 호환: pooler 있으면 사용, 없으면 [CLS]
        if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
            pooled = outputs.pooler_output                       # (B, 768)
        else:
            pooled = outputs[0][:, 0]                            # last_hidden_state[:, 0]

        logits = self.classifier(self.dropout(pooled))           # (B, num_classes)

        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return logits, loss
        return logits, None


In [22]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(data.dropna(subset =['text','label']),
    test_size=0.3, random_state=42, stratify=data['label']                                      
)

import numpy as np
counts = train_df["label"].value_counts().sort_index()
N, K = counts.sum(), counts.shape[0]
class_weights = torch.tensor([float(N/(counts[i]*K)) for i in range(K)], dtype=torch.float32).to(device)
print("class_weights:", class_weights)
print(counts)

KeyError: ['label']

In [ ]:
from torch.utils.data import Dataset, DataLoader

# 한 문장씩 KoBert토크나이즈
class IntentDataset(Dataset) : 
    def __init__(self, data, tokenizer, max_len) : 
        self.texts = data['text'].astype(str).tolist()
        self.labels = data['label'].astype(int).tolist()
        self.tok = tokenizer
        self.max_len = max_len

    def __len__(self) : return len(self.texts)

    def __getitem__(self, idx) :
        enc = self.tok(
            self.texts[idx],
            padding = 'max_length',
            truncation = True,
            max_length = self.max_len,
            return_tensors = 'pt'
        )

        item = {
            "input_ids" : enc["input_ids"].squeeze(0),
            "attention_mask" : enc["attention_mask"].squeeze(0),
            "labels" : torch.tensor(self.labels[idx], dtype=torch.long)
        }

        if "token_type_ids" in enc : 
            item["token_type_ids"] = enc["token_type_ids"].squeeze(0)
        else : 
            item['token_type_ids'] = None
        return item

# 미니배치 단위로 텐서 묶음
train_loader = DataLoader(IntentDataset(train_df, tokenizer, MAX_LEN), batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
valid_loader = DataLoader(IntentDataset(valid_df, tokenizer, MAX_LEN), batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

NameError: name 'train_df' is not defined

In [27]:
# 모델 옵티마이저, 스케쥴러 세팅

from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup

model = BertClassifier(
    bert = bertmodel,
    hidden_size=768,
    num_classes = NUM_LABELS,
    dr_rate=DROPOUT,
    class_weights=class_weights
).to(device)

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

total_steps = len(train_loader) * EPOCHS
warmup_steps = int(total_steps * 0.06)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

NameError: name 'BertClassifier' is not defined

In [23]:
# 학습/검증 루프

import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score, classification_report

def train_one_epoch(model, loader):
    model.train()
    total_loss, all_p, all_y = 0.0, [], []
    for batch in tqdm(loader, desc="Train"):
        input_ids      = batch["input_ids"].to(device, non_blocking=True)
        attention_mask = batch["attention_mask"].to(device, non_blocking=True)
        token_type_ids = batch["token_type_ids"]
        if token_type_ids is not None:
            token_type_ids = token_type_ids.to(device, non_blocking=True)
        labels         = batch["labels"].to(device, non_blocking=True)

        optimizer.zero_grad()
        logits, loss = model(input_ids, attention_mask, token_type_ids, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        all_p.extend(torch.argmax(logits, 1).detach().cpu().tolist())
        all_y.extend(labels.detach().cpu().tolist())
    return total_loss/len(loader), accuracy_score(all_y, all_p), f1_score(all_y, all_p, average="macro")

@torch.no_grad()
def evaluate(model, loader, target_names=None):
    model.eval()
    total_loss, all_p, all_y = 0.0, [], []
    for batch in tqdm(loader, desc="Valid"):
        input_ids      = batch["input_ids"].to(device, non_blocking=True)
        attention_mask = batch["attention_mask"].to(device, non_blocking=True)
        token_type_ids = batch["token_type_ids"]
        if token_type_ids is not None:
            token_type_ids = token_type_ids.to(device, non_blocking=True)
        labels         = batch["labels"].to(device, non_blocking=True)

        logits, loss = model(input_ids, attention_mask, token_type_ids, labels)
        if loss is not None:
            total_loss += loss.item()
        all_p.extend(torch.argmax(logits, 1).detach().cpu().tolist())
        all_y.extend(labels.detach().cpu().tolist())
    rep = classification_report(all_y, all_p, target_names=target_names, digits=4)
    return total_loss/len(loader), accuracy_score(all_y, all_p), f1_score(all_y, all_p, average="macro"), rep

best_f1, patience = 0.0, 0
label_dict = { 'Communication':0, 'Teamwork_Leadership':1, 'Integrity':2, 'Adaptability':3, 'Job_Competency':4 }
label_names = [k for k,_ in sorted(label_dict.items(), key=lambda x: x[1])]

for epoch in range(1, EPOCHS+1):
    print(f"\n==== EPOCH {epoch}/{EPOCHS} ====")
    tr_loss, tr_acc, tr_f1 = train_one_epoch(model, train_loader)
    val_loss, val_acc, val_f1, val_rep = evaluate(model, valid_loader, target_names=label_names)

    print(f"[Train] loss={tr_loss:.4f} acc={tr_acc:.4f} f1={tr_f1:.4f}")
    print(f"[Valid] loss={val_loss:.4f} acc={val_acc:.4f} f1={val_f1:.4f}")
    print(val_rep)

    if val_f1 > best_f1:
        best_f1, patience = val_f1, 0
        os.makedirs("./kobert_intent_model", exist_ok=True)
        torch.save(model.state_dict(), "./kobert_intent_model/model.pt")
        with open("./kobert_intent_model/label_map.txt", "w", encoding="utf-8") as f:
            for k,v in sorted(label_dict.items(), key=lambda x: x[1]):
                f.write(f"{v}\t{k}\n")
        print(f">>> Best saved. macroF1={best_f1:.4f}")
    else:
        patience += 1
        if patience >= PATIENCE:
            print("Early stopping.")
            break



==== EPOCH 1/10 ====


Valid: 100%|█████████████████| 305/305 [01:21<00:00,  3.76it/s]


[Train] loss=0.9754 acc=0.6536 f1=0.5603
[Valid] loss=0.5931 acc=0.7989 f1=0.7344
                     precision    recall  f1-score   support

      Communication     0.9522    0.7889    0.8629      5989
Teamwork_Leadership     0.5684    0.7871    0.6601       512
          Integrity     0.8406    0.8318    0.8362      1344
       Adaptability     0.4963    0.7948    0.6111       765
     Job_Competency     0.6131    0.8208    0.7019      1133

           accuracy                         0.7989      9743
          macro avg     0.6941    0.8047    0.7344      9743
       weighted avg     0.8414    0.7989    0.8101      9743

>>> Best saved. macroF1=0.7344

==== EPOCH 2/10 ====


Valid: 100%|█████████████████| 305/305 [02:28<00:00,  2.05it/s]


[Train] loss=0.5071 acc=0.8389 f1=0.7790
[Valid] loss=0.5759 acc=0.8246 f1=0.7626
                     precision    recall  f1-score   support

      Communication     0.9518    0.8243    0.8835      5989
Teamwork_Leadership     0.7173    0.7188    0.7180       512
          Integrity     0.7368    0.8936    0.8077      1344
       Adaptability     0.5417    0.7974    0.6452       765
     Job_Competency     0.7133    0.8102    0.7587      1133

           accuracy                         0.8246      9743
          macro avg     0.7322    0.8089    0.7626      9743
       weighted avg     0.8499    0.8246    0.8311      9743

>>> Best saved. macroF1=0.7626

==== EPOCH 3/10 ====


...

[Train] loss=0.0251 acc=0.9921 f1=0.9897
[Valid] loss=1.2161 acc=0.8720 f1=0.8090
                     precision    recall  f1-score   support

      Communication     0.9034    0.9320    0.9175      5989
Teamwork_Leadership     0.7490    0.7520    0.7505       512
          Integrity     0.8668    0.8571    0.8620      1344
       Adaptability     0.7828    0.7020    0.7402       765
     Job_Competency     0.8116    0.7414    0.7749      1133

           accuracy                         0.8720      9743
          macro avg     0.8227    0.7969    0.8090      9743
       weighted avg     0.8701    0.8720    0.8705      9743

>>> Best saved. macroF1=0.8090

==== EPOCH 9/10 ====


Valid: 100%|█████████████████| 305/305 [02:26<00:00,  2.08it/s]


[Train] loss=0.0148 acc=0.9949 f1=0.9934
[Valid] loss=1.2377 acc=0.8726 f1=0.8109
                     precision    recall  f1-score   support

      Communication     0.9039    0.9324    0.9179      5989
Teamwork_Leadership     0.7609    0.7520    0.7564       512
          Integrity     0.8791    0.8385    0.8583      1344
       Adaptability     0.7679    0.7137    0.7398       765
     Job_Competency     0.8068    0.7590    0.7822      1133

           accuracy                         0.8726      9743
          macro avg     0.8237    0.7991    0.8109      9743
       weighted avg     0.8710    0.8726    0.8714      9743

>>> Best saved. macroF1=0.8109

==== EPOCH 10/10 ====


Valid: 100%|█████████████████| 305/305 [01:21<00:00,  3.75it/s]


[Train] loss=0.0110 acc=0.9964 f1=0.9953
[Valid] loss=1.2492 acc=0.8727 f1=0.8107
                     precision    recall  f1-score   support

      Communication     0.9052    0.9312    0.9180      5989
Teamwork_Leadership     0.7644    0.7539    0.7591       512
          Integrity     0.8702    0.8527    0.8613      1344
       Adaptability     0.7773    0.6980    0.7355       765
     Job_Competency     0.8015    0.7590    0.7797      1133

           accuracy                         0.8727      9743
          macro avg     0.8237    0.7990    0.8107      9743
       weighted avg     0.8709    0.8727    0.8714      9743



## 모델 실행

In [1]:
import torch
import torch.nn as nn
from kobert_transformers import get_kobert_model, get_tokenizer
import re

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 클래스 정의 (학습 때와 동일)
class BertClassifier(nn.Module):
    def __init__(self, 
                 bert,
                 hidden_size: int = 768,
                 num_classes: int = 5,
                 dr_rate: float = 0.3,
                 class_weights: torch.Tensor | None = None):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=dr_rate) if dr_rate and dr_rate > 0 else nn.Identity()
        self.classifier = nn.Linear(hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights) if class_weights is not None else nn.CrossEntropyLoss()
    
    def forward(self,
                input_ids: torch.Tensor,
                attention_mask: torch.Tensor | None = None,
                token_type_ids: torch.Tensor | None = None,
                labels: torch.Tensor | None = None):
        
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        
        if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
            pooled = outputs.pooler_output
        else:
            pooled = outputs[0][:, 0]

        logits = self.classifier(self.dropout(pooled))

        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return logits, loss
        return logits, None


def load_model(model_path, label_map_path):
    """학습된 모델과 라벨 매핑 로드"""
    
    # 라벨 매핑 로드
    label_map = {}
    with open(label_map_path, 'r', encoding='utf-8') as f:
        for line in f:
            idx, label = line.strip().split('\t')
            label_map[int(idx)] = label
    
    # 모델 초기화
    bertmodel = get_kobert_model()
    model = BertClassifier(
        bert=bertmodel,
        hidden_size=768,
        num_classes=5,
        dr_rate=0.3,
        class_weights=None
    ).to(device)
    
    # 학습된 가중치 로드 (strict=False로 설정하여 loss_fn.weight 무시)
    state_dict = torch.load(model_path, map_location=device)
    
    # loss_fn.weight 제거 (있는 경우)
    if 'loss_fn.weight' in state_dict:
        del state_dict['loss_fn.weight']
    
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    
    return model, label_map


def split_sentences(text):
    """문단을 문장으로 분리"""
    # 줄바꿈 제거 및 공백 정리
    text = re.sub(r'\s+', ' ', text.strip())
    
    # 문장 분리 (마침표, 물음표, 느낌표 기준)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    # 빈 문장 제거
    sentences = [s.strip() for s in sentences if s.strip()]
    
    return sentences


@torch.no_grad()
def predict_intent(model, tokenizer, sentence, label_map, max_len=502):
    """단일 문장의 의도 예측"""
    model.eval()
    
    # 토크나이징
    encoding = tokenizer(
        sentence,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    token_type_ids = encoding.get('token_type_ids')
    if token_type_ids is not None:
        token_type_ids = token_type_ids.to(device)
    
    # 예측
    logits, _ = model(input_ids, attention_mask, token_type_ids)
    probs = torch.softmax(logits, dim=1)
    pred_idx = torch.argmax(probs, dim=1).item()
    confidence = probs[0][pred_idx].item()
    
    return label_map[pred_idx], confidence, probs[0].cpu().numpy()


def analyze_paragraph(model, tokenizer, paragraph, label_map):
    """문단 전체 분석"""
    sentences = split_sentences(paragraph)
    
    print("=" * 80)
    print("📝 문단 분석 결과")
    print("=" * 80)
    print(f"\n총 {len(sentences)}개의 문장이 발견되었습니다.\n")
    
    results = []
    
    for i, sentence in enumerate(sentences, 1):
        intent, confidence, probs = predict_intent(model, tokenizer, sentence, label_map)
        results.append({
            'sentence_num': i,
            'sentence': sentence,
            'intent': intent,
            'confidence': confidence,
            'probabilities': probs
        })
        
        print(f"[문장 {i}]")
        print(f"내용: {sentence}")
        print(f"분류: {intent} (확신도: {confidence:.2%})")
        
        # 모든 클래스별 확률 출력
        print("세부 확률:")
        for idx, prob in enumerate(probs):
            print(f"  - {label_map[idx]}: {prob:.2%}")
        print("-" * 80)
    
    return results


def print_summary(results):
    """요약 통계 출력"""
    print("\n" + "=" * 80)
    print("📊 분석 요약")
    print("=" * 80)
    
    intent_counts = {}
    for result in results:
        intent = result['intent']
        intent_counts[intent] = intent_counts.get(intent, 0) + 1
    
    print("\n의도별 문장 수:")
    for intent, count in sorted(intent_counts.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / len(results)) * 100
        print(f"  {intent}: {count}개 ({percentage:.1f}%)")
    
    avg_confidence = sum(r['confidence'] for r in results) / len(results)
    print(f"\n평균 확신도: {avg_confidence:.2%}")


# 메인 실행 코드
if __name__ == "__main__":
    # 모델 및 토크나이저 로드
    MODEL_PATH = "/home/j-k13b204/S13P31B204/model_test/kobert_intent_model/model.pt"
    LABEL_MAP_PATH = "/home/j-k13b204/S13P31B204/model_test/kobert_intent_model/label_map.txt"
    
    print("모델을 로드하는 중...")
    tokenizer = get_tokenizer()
    model, label_map = load_model(MODEL_PATH, LABEL_MAP_PATH)
    print("✓ 모델 로드 완료!\n")
    
    # 분석할 문단
    sentence = """
    최근에는 'VR 기반 면접 AI 분석 시스템'을 직접 구현하며
    데이터 수집부터 모델 학습까지의 전 과정을 수행했습니다.
    AI-Hub 면접 데이터셋을 가공하고, KoBERT 모델을 fine-tuning하여
    지원자의 답변을 5대 NCS 역량(Communication, Integrity, Adaptability, Teamwork, Job Competency) 으로 분류했습니다.

    또한 FastAPI와 LangChain을 이용해 GPT API와 연동한 면접 피드백 자동 생성 시스템을 개발하며
    AI가 단순한 분석 도구가 아닌 경험을 개선하는 서비스가 될 수 있음을 실감했습니다.
    이 경험을 통해 데이터 분석과 AI 서비스 기획, 그리고 사용자 경험을 연결하는 시각을 키웠습니다.
    """
    
    # 분석 실행
    results = analyze_paragraph(model, tokenizer, sentence, label_map)
    
    # 요약 출력
    print_summary(results)

모델을 로드하는 중...
✓ 모델 로드 완료!

📝 문단 분석 결과

총 4개의 문장이 발견되었습니다.

[문장 1]
내용: 최근에는 'VR 기반 면접 AI 분석 시스템'을 직접 구현하며 데이터 수집부터 모델 학습까지의 전 과정을 수행했습니다.
분류: Job_Competency (확신도: 99.95%)
세부 확률:
  - Communication: 0.02%
  - Teamwork_Leadership: 0.00%
  - Integrity: 0.01%
  - Adaptability: 0.01%
  - Job_Competency: 99.95%
--------------------------------------------------------------------------------
[문장 2]
내용: AI-Hub 면접 데이터셋을 가공하고, KoBERT 모델을 fine-tuning하여 지원자의 답변을 5대 NCS 역량(Communication, Integrity, Adaptability, Teamwork, Job Competency) 으로 분류했습니다.
분류: Teamwork_Leadership (확신도: 98.60%)
세부 확률:
  - Communication: 0.02%
  - Teamwork_Leadership: 98.60%
  - Integrity: 0.94%
  - Adaptability: 0.03%
  - Job_Competency: 0.41%
--------------------------------------------------------------------------------
[문장 3]
내용: 또한 FastAPI와 LangChain을 이용해 GPT API와 연동한 면접 피드백 자동 생성 시스템을 개발하며 AI가 단순한 분석 도구가 아닌 경험을 개선하는 서비스가 될 수 있음을 실감했습니다.
분류: Job_Competency (확신도: 62.25%)
세부 확률:
  - Communication: 0.19%
  - Teamwork_Leader

In [ ]:
import torch
import torch.nn as nn
import torch.quantization as quant
from kobert_transformers import get_kobert_model
import os

# ===== 모델 클래스 정의 (원본과 동일) =====
class BertClassifier(nn.Module):
    def __init__(self, 
                 bert,
                 hidden_size: int = 768,
                 num_classes: int = 5,
                 dr_rate: float = 0.3,
                 class_weights: torch.Tensor | None = None):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=dr_rate) if dr_rate and dr_rate > 0 else nn.Identity()
        self.classifier = nn.Linear(hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights) if class_weights is not None else nn.CrossEntropyLoss()
    
    def forward(self,
                input_ids: torch.Tensor,
                attention_mask: torch.Tensor | None = None,
                token_type_ids: torch.Tensor | None = None,
                labels: torch.Tensor | None = None):
        
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        
        if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
            pooled = outputs.pooler_output
        else:
            pooled = outputs[0][:, 0]

        logits = self.classifier(self.dropout(pooled))

        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return logits, loss
        return logits, None


def load_model(model_path):
    """학습된 모델 로드"""
    device = torch.device('cpu')  # 양자화는 CPU에서 진행
    
    # 모델 초기화
    print("모델 구조 초기화 중...")
    bertmodel = get_kobert_model()
    model = BertClassifier(
        bert=bertmodel,
        hidden_size=768,
        num_classes=5,
        dr_rate=0.3,
        class_weights=None
    ).to(device)
    
    # 학습된 가중치 로드
    print("가중치 로드 중...")
    state_dict = torch.load(model_path, map_location=device)
    
    # loss_fn.weight 제거 (있는 경우)
    if 'loss_fn.weight' in state_dict:
        del state_dict['loss_fn.weight']
    
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    
    return model


def quantize_model(model):
    """모델 동적 양자화"""
    print("\n동적 양자화 수행 중...")
    quantized_model = quant.quantize_dynamic(
        model,
        {nn.Linear},  # Linear 레이어 양자화
        dtype=torch.qint8
    )
    return quantized_model


def export_to_onnx(model, output_path, opset_version=14):
    """ONNX 형식으로 변환"""
    print(f"\nONNX 변환 중 (opset_version={opset_version})...")
    
    # 더미 입력 생성
    batch_size = 1
    seq_length = 128
    
    dummy_input = {
        'input_ids': torch.randint(0, 8002, (batch_size, seq_length), dtype=torch.long),
        'attention_mask': torch.ones(batch_size, seq_length, dtype=torch.long),
        'token_type_ids': torch.zeros(batch_size, seq_length, dtype=torch.long)
    }
    
    try:
        torch.onnx.export(
            model,
            (dummy_input['input_ids'], 
             dummy_input['attention_mask'], 
             dummy_input['token_type_ids']),
            output_path,
            export_params=True,
            opset_version=opset_version,
            do_constant_folding=True,
            input_names=['input_ids', 'attention_mask', 'token_type_ids'],
            output_names=['logits'],
            dynamic_axes={
                'input_ids': {0: 'batch_size', 1: 'sequence'},
                'attention_mask': {0: 'batch_size', 1: 'sequence'},
                'token_type_ids': {0: 'batch_size', 1: 'sequence'},
                'logits': {0: 'batch_size'}
            }
        )
        return True
    except Exception as e:
        print(f"ONNX 변환 실패: {e}")
        return False


def print_model_info(path, name):
    """모델 정보 출력"""
    if os.path.exists(path):
        size_mb = os.path.getsize(path) / (1024**2)
        print(f"  ✓ {name}: {size_mb:.2f} MB")
        return size_mb
    else:
        print(f"  ✗ {name}: 파일 없음")
        return 0


# ===== 메인 실행 =====
if __name__ == "__main__":
    # 경로 설정
    MODEL_PATH = "/home/j-k13b204/S13P31B204/model_test/kobert_intent_model/model.pt"
    OUTPUT_DIR = "/home/j-k13b204/S13P31B204/model_test/kobert_intent_model/quantize"
    
    QUANTIZED_PT_PATH = os.path.join(OUTPUT_DIR, "model_quantized.pt")
    QUANTIZED_ONNX_PATH = os.path.join(OUTPUT_DIR, "model_quantized.onnx")
    ORIGINAL_ONNX_PATH = os.path.join(OUTPUT_DIR, "model_original.onnx")
    
    print("="*80)
    print("KoBERT Intent Model 양자화 및 ONNX 변환")
    print("="*80)
    
    # 1. 원본 모델 로드
    print("\n[1단계] 원본 모델 로드")
    model = load_model(MODEL_PATH)
    original_size = print_model_info(MODEL_PATH, "원본 모델")
    
    # 2. 양자화
    print("\n[2단계] 모델 양자화")
    quantized_model = quantize_model(model)
    
    # 3. 양자화 모델 저장 (.pt)
    print(f"\n양자화된 PyTorch 모델 저장 중...")
    torch.save(quantized_model, QUANTIZED_PT_PATH)
    quantized_size = print_model_info(QUANTIZED_PT_PATH, "양자화 모델 (.pt)")
    
    # 4. ONNX 변환 (양자화 모델)
    print("\n[3단계] 양자화 모델 ONNX 변환")
    success = export_to_onnx(quantized_model, QUANTIZED_ONNX_PATH)
    
    if not success:
        print("\n⚠️  양자화 모델 ONNX 변환 실패. 원본 모델로 시도합니다...")
        print("\n[대안] 원본 모델 ONNX 변환")
        export_to_onnx(model, ORIGINAL_ONNX_PATH)
        print("\n💡 원본 모델을 ONNX로 변환했습니다.")
        print("   onnxruntime을 사용하여 ONNX 모델을 양자화할 수 있습니다:")
        print(f"   python -m onnxruntime.quantization.preprocess --input {ORIGINAL_ONNX_PATH} --output {QUANTIZED_ONNX_PATH}")
    
    # 결과 요약
    print("\n" + "="*80)
    print("📊 결과 요약")
    print("="*80)
    print("\n생성된 파일:")
    original_size = print_model_info(MODEL_PATH, "원본 모델")
    quantized_pt_size = print_model_info(QUANTIZED_PT_PATH, "양자화 모델 (.pt)")
    
    if os.path.exists(QUANTIZED_ONNX_PATH):
        onnx_size = print_model_info(QUANTIZED_ONNX_PATH, "양자화 모델 (.onnx)")
        print(f"\n✅ ONNX 변환 성공!")
    elif os.path.exists(ORIGINAL_ONNX_PATH):
        onnx_size = print_model_info(ORIGINAL_ONNX_PATH, "원본 모델 (.onnx)")
        print(f"\n⚠️  원본 ONNX만 생성됨 (양자화는 별도 도구 필요)")
    
    if quantized_pt_size > 0 and original_size > 0:
        compression_ratio = original_size / quantized_pt_size
        size_reduction = ((original_size - quantized_pt_size) / original_size) * 100
        print(f"\n압축률: {compression_ratio:.2f}x ({size_reduction:.1f}% 감소)")
    
    print("\n" + "="*80)
    print("✅ 완료!")
    print("="*80)

In [2]:
import torch
import torch.nn as nn
from kobert_transformers import get_tokenizer
import re

# ⚠️ 양자화 모델은 CPU 전용!
device = torch.device("cpu")

class BertClassifier(nn.Module):
    def __init__(self, 
                 bert,
                 hidden_size: int = 768,
                 num_classes: int = 5,
                 dr_rate: float = 0.3,
                 class_weights: torch.Tensor | None = None):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=dr_rate) if dr_rate and dr_rate > 0 else nn.Identity()
        self.classifier = nn.Linear(hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights) if class_weights is not None else nn.CrossEntropyLoss()
    
    def forward(self,
                input_ids: torch.Tensor,
                attention_mask: torch.Tensor | None = None,
                token_type_ids: torch.Tensor | None = None,
                labels: torch.Tensor | None = None):
        
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        
        if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
            pooled = outputs.pooler_output
        else:
            pooled = outputs[0][:, 0]

        logits = self.classifier(self.dropout(pooled))

        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return logits, loss
        return logits, None

def load_quantized_model(model_path, label_map_path):
    """양자화된 모델과 라벨 매핑 로드 (CPU 전용)"""
    
    print("⚠️  양자화 모델은 CPU에서만 실행됩니다.")
    print("💡 GPU를 사용하려면 원본 모델(model.pt)을 사용하세요.\n")
    
    # 라벨 매핑 로드
    label_map = {}
    with open(label_map_path, 'r', encoding='utf-8') as f:
        for line in f:
            idx, label = line.strip().split('\t')
            label_map[int(idx)] = label
    
    # 양자화된 모델 직접 로드 (모델 클래스 정의 불필요!)
    print("양자화 모델 로드 중...")
    model = torch.load(model_path, map_location='cpu')
    model.eval()
    
    print("✓ 모델 로드 완료 (CPU 모드)\n")
    return model, label_map


def split_sentences(text):
    """문단을 문장으로 분리"""
    # 줄바꿈 제거 및 공백 정리
    text = re.sub(r'\s+', ' ', text.strip())
    
    # 문장 분리 (마침표, 물음표, 느낌표 기준)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    # 빈 문장 제거
    sentences = [s.strip() for s in sentences if s.strip()]
    
    return sentences


@torch.no_grad()
def predict_intent(model, tokenizer, sentence, label_map, max_len=502):
    """단일 문장의 의도 예측"""
    model.eval()
    
    # 토크나이징
    encoding = tokenizer(
        sentence,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    
    # CPU에서 실행 (.to(device) 불필요 - 이미 CPU)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    token_type_ids = encoding.get('token_type_ids')
    
    # 예측
    logits, _ = model(input_ids, attention_mask, token_type_ids)
    probs = torch.softmax(logits, dim=1)
    pred_idx = torch.argmax(probs, dim=1).item()
    confidence = probs[0][pred_idx].item()
    
    return label_map[pred_idx], confidence, probs[0].numpy()


def analyze_paragraph(model, tokenizer, paragraph, label_map):
    """문단 전체 분석"""
    sentences = split_sentences(paragraph)
    
    print("=" * 80)
    print("📝 문단 분석 결과")
    print("=" * 80)
    print(f"\n총 {len(sentences)}개의 문장이 발견되었습니다.\n")
    
    results = []
    
    for i, sentence in enumerate(sentences, 1):
        intent, confidence, probs = predict_intent(model, tokenizer, sentence, label_map)
        results.append({
            'sentence_num': i,
            'sentence': sentence,
            'intent': intent,
            'confidence': confidence,
            'probabilities': probs
        })
        
        print(f"[문장 {i}]")
        print(f"내용: {sentence}")
        print(f"분류: {intent} (확신도: {confidence:.2%})")
        
        # 모든 클래스별 확률 출력
        print("세부 확률:")
        for idx, prob in enumerate(probs):
            print(f"  - {label_map[idx]}: {prob:.2%}")
        print("-" * 80)
    
    return results


def print_summary(results):
    """요약 통계 출력"""
    print("\n" + "=" * 80)
    print("📊 분석 요약")
    print("=" * 80)
    
    intent_counts = {}
    for result in results:
        intent = result['intent']
        intent_counts[intent] = intent_counts.get(intent, 0) + 1
    
    print("\n의도별 문장 수:")
    for intent, count in sorted(intent_counts.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / len(results)) * 100
        print(f"  {intent}: {count}개 ({percentage:.1f}%)")
    
    avg_confidence = sum(r['confidence'] for r in results) / len(results)
    print(f"\n평균 확신도: {avg_confidence:.2%}")


# 메인 실행 코드
if __name__ == "__main__":
    print("="*80)
    print("🚀 양자화 KoBERT 모델 추론 (CPU 최적화)")
    print("="*80)
    print()
    
    # 모델 및 토크나이저 로드
    MODEL_PATH = "/home/j-k13b204/S13P31B204/model_test/kobert_intent_model/quantize/model_quantized.pt"
    LABEL_MAP_PATH = "/home/j-k13b204/S13P31B204/model_test/kobert_intent_model/quantize/label_map.txt"
    
    print("모델을 로드하는 중...")
    tokenizer = get_tokenizer()
    model, label_map = load_quantized_model(MODEL_PATH, LABEL_MAP_PATH)
    
    # 분석할 문단
    sentence = """
    최근에는 'VR 기반 면접 AI 분석 시스템'을 직접 구현하며
    데이터 수집부터 모델 학습까지의 전 과정을 수행했습니다.
    AI-Hub 면접 데이터셋을 가공하고, KoBERT 모델을 fine-tuning하여
    지원자의 답변을 5대 NCS 역량(Communication, Integrity, Adaptability, Teamwork, Job Competency) 으로 분류했습니다.

    또한 FastAPI와 LangChain을 이용해 GPT API와 연동한 면접 피드백 자동 생성 시스템을 개발하며
    AI가 단순한 분석 도구가 아닌 경험을 개선하는 서비스가 될 수 있음을 실감했습니다.
    이 경험을 통해 데이터 분석과 AI 서비스 기획, 그리고 사용자 경험을 연결하는 시각을 키웠습니다.
    """
    
    # 분석 실행
    results = analyze_paragraph(model, tokenizer, sentence, label_map)
    
    # 요약 출력
    print_summary(results)
    
    print("\n" + "="*80)
    print("💡 팁:")
    print("  - 양자화 모델은 원본 대비 약 4배 작고 2-3배 빠릅니다")
    print("  - CPU에서 최적화되어 있어 GPU 없이도 빠른 추론 가능")
    print("  - FastAPI 등 프로덕션 배포에 적합합니다")
    print("="*80)

🚀 양자화 KoBERT 모델 추론 (CPU 최적화)

모델을 로드하는 중...


⚠️  양자화 모델은 CPU에서만 실행됩니다.
💡 GPU를 사용하려면 원본 모델(model.pt)을 사용하세요.

양자화 모델 로드 중...
✓ 모델 로드 완료 (CPU 모드)

📝 문단 분석 결과

총 4개의 문장이 발견되었습니다.

[문장 1]
내용: 최근에는 'VR 기반 면접 AI 분석 시스템'을 직접 구현하며 데이터 수집부터 모델 학습까지의 전 과정을 수행했습니다.
분류: Job_Competency (확신도: 99.94%)
세부 확률:
  - Communication: 0.02%
  - Teamwork_Leadership: 0.01%
  - Integrity: 0.01%
  - Adaptability: 0.02%
  - Job_Competency: 99.94%
--------------------------------------------------------------------------------
[문장 2]
내용: AI-Hub 면접 데이터셋을 가공하고, KoBERT 모델을 fine-tuning하여 지원자의 답변을 5대 NCS 역량(Communication, Integrity, Adaptability, Teamwork, Job Competency) 으로 분류했습니다.
분류: Teamwork_Leadership (확신도: 84.76%)
세부 확률:
  - Communication: 0.22%
  - Teamwork_Leadership: 84.76%
  - Integrity: 7.38%
  - Adaptability: 1.63%
  - Job_Competency: 6.02%
--------------------------------------------------------------------------------
[문장 3]
내용: 또한 FastAPI와 LangChain을 이용해 GPT API와 연동한 면접 피드백 자동 생성 시스템을 개발하며 AI가 단순한 분석 도구가 아닌 경험을 개선하는 서비스가 될 수 있음을 실감했습니다.
분류: Job_Com